In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

import pickle

In [2]:
data_white = Path('../UCI_5000/winequality-white.csv')
df_white = pd.read_csv(data_white, sep = ';')
df_white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
data_red = Path('../UCI_5000/winequality-red.csv')
df_red = pd.read_csv(data_red, sep = ';')
df_red.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
df_white['quality'].value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [12]:
y = df_red['quality']
X = df_red.drop(columns = 'quality')
# X = df_white[['fixed acidity', 'residual sugar','alcohol']] # 'volatile acidity' 'pH'

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(1199, 11)

In [14]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [15]:
LogR = LogisticRegression(solver='sag',
                                max_iter=500,
                                random_state=1)
LogR

LogisticRegression(max_iter=500, random_state=1, solver='sag')

In [16]:
LogR.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=500, random_state=1, solver='sag')

In [17]:
# LogR.predict([[0.5],[0.5],[0.5]])

In [18]:
# Make predictions on the test set
y_pred = LogR.predict(X_test_scaled)

# Calculate the R-squared value
r_squared = r2_score(y_test, y_pred)
print(r_squared)

0.24496321121768938


In [19]:
# export ~dummy model for testing the app 
pickle.dump(LogR, open('LogRDummyModelRed.pkl', 'wb'))
# joblib.dump(LogR)

In [51]:
# At 500 iterations 
# Make predictions on the test set
y_pred = LogR.predict(X_test)

# Calculate the R-squared value
r_squared = r2_score(y_test, y_pred)
print(r_squared)
print(accuracy_score(y_test, y_pred))

-5.35659903874782
0.05142857142857143


C:\Users\Benjamin\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [52]:
X_train_polystd

array([[ 0.        , -0.90858237,  1.69568748, ...,  0.70863396,
         0.47071645, -0.96740188],
       [ 0.        , -0.06539301,  1.71537104, ...,  1.42647634,
         1.95131138, -0.2115916 ],
       [ 0.        , -0.78812674,  1.18391509, ...,  0.60208499,
         0.83073025, -0.57009509],
       ...,
       [ 0.        , -0.30630425,  1.67600393, ...,  0.69297534,
         0.45930498, -0.96740188],
       [ 0.        , -0.06539301, -0.72538959, ..., -0.42189321,
        -0.82526425, -0.67177038],
       [ 0.        ,  0.05506262, -0.27266786, ..., -0.14357235,
         0.68554871,  1.3604719 ]])

In [48]:
# Daniel
from sklearn.preprocessing import PolynomialFeatures

# Create 8th degree polynomial matrix
poly = PolynomialFeatures(5)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

X_train_polystd = scaler.fit_transform(X_train_poly)
X_test_polystd = scaler.transform(X_test_poly)

# Perform linear regression
reg = LinearRegression().fit(X_train_polystd, y_train)

y_pred_train = reg.predict(X_train_polystd)
y_pred_test = reg.predict(X_test_polystd)

print('RMSE Train: ', mean_squared_error(y_train, y_pred_train, squared=False))
print('RMSE Test: ', mean_squared_error(y_test, y_pred_test, squared=False))

RMSE Train:  0.7737320507801323
RMSE Test:  0.7638161951628708


In [51]:
# export ~dummy model for testing the app 
pickle.dump(reg, open('PolyDummyModel.sav', 'wb'))

In [41]:
print('RMSE Train: ', mean_squared_error(y_train, y_pred_train, squared=False))
print('RMSE Val: ', mean_squared_error(y_test, y_pred_val, squared=False))

RMSE Train:  1.5814786049692585e-10
RMSE Val:  337.817427199671
-145677.4029292947


In [44]:
accuracy_score(y_test, y_pred_val)

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets